<a href="https://colab.research.google.com/github/dataengs/pyspark-lateral-view/blob/main/pyspark_lateral_view.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#pip install pyspark

In [28]:
#importa bibliotecas
from pyspark.sql import SparkSession
#cria sessao spark
spark = SparkSession.builder.appName('pyspark-lateral-view').getOrCreate()
#cria uma lista de vendas
dataset = [("Jose", ["Camisa"] ,10, 202201),
           ("Marcos", ["Camisa","Bermuda","Jaqueta"], 25, 202201),
           ("João", ["Sapato", "Bermuda"], 17, 202201) 
          ]
#cria um schema manual para criar o dataframe
columns = ["nomeVendedor","produto","qtdVendas","mesAnoVenda"]
#cria o dataframe
df = spark.createDataFrame(data=dataset, schema = columns)

In [30]:
df.show(truncate = False)

+------------+--------------------------+---------+-----------+
|nomeVendedor|produto                   |qtdVendas|mesAnoVenda|
+------------+--------------------------+---------+-----------+
|Jose        |[Camisa]                  |10       |202201     |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |
|João        |[Sapato, Bermuda]         |17       |202201     |
+------------+--------------------------+---------+-----------+



In [31]:
#Criando uma view sql
df.createOrReplaceTempView("sql_view")

In [32]:
spark.sql('''SELECT * FROM sql_view''').show(truncate = False)

+------------+--------------------------+---------+-----------+
|nomeVendedor|produto                   |qtdVendas|mesAnoVenda|
+------------+--------------------------+---------+-----------+
|Jose        |[Camisa]                  |10       |202201     |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |
|João        |[Sapato, Bermuda]         |17       |202201     |
+------------+--------------------------+---------+-----------+



In [38]:
#Executando a cláusula LATERAL VIEW em conjunto com o EXPLODE
#onde irá gerar uma tabela contendo uma ou mais linhas.
#nesse exemplo aplicará as linhas do array a cada linha de saída original
df_result = spark.sql('''SELECT * FROM sql_view
                      LATERAL VIEW EXPLODE(ARRAY(20, 40)) AS a_category
                      LATERAL VIEW EXPLODE(ARRAY(41, 80)) AS b_category''')

In [39]:
df_result.show(truncate=False)

+------------+--------------------------+---------+-----------+----------+----------+
|nomeVendedor|produto                   |qtdVendas|mesAnoVenda|a_category|b_category|
+------------+--------------------------+---------+-----------+----------+----------+
|Jose        |[Camisa]                  |10       |202201     |20        |41        |
|Jose        |[Camisa]                  |10       |202201     |20        |80        |
|Jose        |[Camisa]                  |10       |202201     |40        |41        |
|Jose        |[Camisa]                  |10       |202201     |40        |80        |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |20        |41        |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |20        |80        |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |40        |41        |
|Marcos      |[Camisa, Bermuda, Jaqueta]|25       |202201     |40        |80        |
|João        |[Sapato, Bermuda]         |17       |202